In [1]:
import numpy as np
from tests import generate_regression_data, test_regression_model, test_knn_model

## Реализация метода линейной регрессии

Требования:

* l1-регуляризация
* l2-регуляризация
* При обучении данные подаются в SGD с помощью итератора
* Для нахождения весов используется стохастический градиентный спуск
* Каждый шаг градиентного спуска использует ровно один обучающий пример
* Стохастический градиентный спуск использует приближенно вычисленный градиент

### Необходимо реализовать следующий (-ие) класс (-ы)

In [2]:
class Regression(object):
    '''Класс для предсказания действительно-значного выхода по входу - вектору из R^n. 
    Используется линейная регрессия, то есть если вход X \in R^n, вектор весов W \in R^{n+1},
    то значение регрессии - это [X' 1] * W, то есть y = x1*w1 + x2*w2 + xn*wn + wn+1.
    Обучение - подгонка весов W - будет вестись на парах (x, y).
    
    Параметры
    ----------
    sgd : объект класса SGD
    trainiterator: объект класса TrainIterator
    n_epoch : количество эпох обучения (default = 1)
    '''
    def __init__(self, sgd, trainiterator, n_epoch=1):
        self.sgd = sgd
        self.trainiterator = trainiterator
        self.n_epoch = n_epoch
     
    def fit(self, X, y):
        '''Обучение модели.
        
        Парметры
        ----------
        X : двумерный массив признаков размера n_samples x n_features
        y : массив/список правильных значений размера n_samples
        
        Выход
        -------
        Метод обучает веса W
        '''
        X = np.array(X)
        y = np.array(y)
        
        self.W = np.zeros(X.shape[1])
        for a, b in self.trainiterator:
            self.W = self.sgd.step(a, b, self.W)
        
    def predict(self, X):
        """ Предсказание выходного значения для входных векторов
        
        Параметры
        ----------
        X : двумерный массив признаков размера n_samples x n_features
        
        Выход
        -------
        y : Массив размера n_samples
        """
        X = np.array(X)
        
        return X @ self.W
        
    def score(self, y_gt, y_pred):
        """Возвращает точность регрессии в виде (1 - u/v),
        где u - суммарный квадрат расхождения y_gt с y_pred,
        v - суммарный квадрат расхождения y_gt с матожиданием y_gt
        
        Параметры
        ----------
        y_gt : массив/список правильных значений размера n_samples
        y_pred : массив/список предсказанных значений размера n_samples
        
        Выход
        -------
        accuracy - точность регрессии
        """
        y_gt = np.array(y_gt)
        y_pred = np.array(y_pred)
        
        return 1 - sum((y_gt - y_pred) ** 2) / (sum((y_gt - y_gt.mean()) ** 2) if (y_gt.size > 1) else 1)

#### Шаг градиентного спуска
В функцию `step` передаётся набор признаков одного объекта (переменная `X`), правильное значение (переменная `y`), соответствующее этому объекту из тренировочной выборки, и массив весов (переменная `W`). Функция `step` возвращает обновлённые веса, вычисленные с использованием предыдущих значений весов `W` и градиента от функции потерь `self.grad.grad` по этим старым весам.

In [3]:
class SGD(object):
    '''Класс для реализации метода стохастического градиентного спуска. 
    
    Параметры
    ----------
    grad : функция вычисления градиента
    alpha : градиентный шаг (default = 1.)
    
    '''
    def __init__(self, grad, alpha=1.):
        self.grad = grad
        self.alpha = alpha
     
    def step(self, X, y, W):
        '''Один шаг градиентного спуска.
        
        Параметры
        ----------
        X : двумерный массив признаков размера n_samples x n_features
        y : массив/список правильных значений размера n_samples
        W : массив весов размера n_weights
        
        Выход
        -------
        Метод возвращает обновленные веса
        '''
        X = np.array(X)
        y = np.array(y)
        W = np.array(W)
        
        return W - self.alpha * self.grad.grad(X, y, W)

#### Градиент
Вычисляется приближенно, численно. В функцию `grad` передаётся набор признаков одного объекта (переменная `X`), правильное значение (переменная `y`) соответствующее этому объекту из тренировочной выборки, и массив весов (переменная `W`). При больших объёмах тренировочных данных затратным по памяти может быть использование итерирования по единичной матрице `np.eye(len(X)) * self.delta`.

In [4]:
class Grad(object):
    '''Класс для вычисления градиента по весам от функции потерь. 
    
    Параметры
    ----------
    loss : функция потерь
    delta : параметр численного дифференцирования (default = 0.000001)    
    '''
    def __init__(self, loss, delta=0.000001):
        self.loss = loss
        self.delta = delta
     
    def grad(self, X, y, W):
        '''Вычисление градиента.
        
        Параметры
        ----------
        X : двумерный массив признаков размера n_samples x n_features
        y : массив/список правильных значений размера n_samples
        W : массив весов размера n_weights
        
        Выход
        -------
        Метод возвращает градиент по весам W в точках X от функции потерь
        '''
        X = np.array(X)
        y = np.array(y)
        W = np.array(W)
        
        loss = self.loss.val(X, y, W)
        gradient = list()
        for d in np.eye(len(X)) * self.delta:
            gradient.append((self.loss.val(X, y, W + d) - loss) / self.delta)
        return np.array(gradient)

#### Функция потерь
В функцию `val` передаётся набор признаков одного объекта (переменная `X`), правильное значение (переменная `y`), соответствующее этому объекту из тренировочной выборки, и массив весов (переменная `W`).

In [5]:
class Loss(object):
    '''Класс для вычисления функции потерь. 
    
    Параметры
    ----------
    l1_coef : коэффициент l1 регуляризации (default = 0)
    l2_coef : коэффициент l2 регуляризации (default = 0)
    '''
    def __init__(self, l1_coef=0, l2_coef=0):
        self.l1 = l1_coef
        self.l2 = l2_coef
     
    def val(self, X, y, W):
        '''Вычисление функции потерь.
        
        Параметры
        ----------
        X : двумерный массив признаков размера n_samples x n_features
        y : массив/список правильных значений размера n_samples
        W : массив весов размера n_weights
        
        Выход
        -------
        Метод возвращает значение функции потерь в точках X
        '''
        X = np.array(X)
        y = np.array(y)
        W = np.array(W)
        
        return (W @ X - y) ** 2 + self.l1 * sum(abs(W)) + self.l2 * sum(W ** 2)

#### Итератор
На каждой итерации возвращает пару: набор значений признаков одного объекта из X и соответствующее ему значение из y.

In [6]:
class TrainIterator(object):
    '''Класс итератора для работы с обучающими данными. 
    
    Параметры
    ----------
    X : двумерный массив признаков размера n_samples x n_features
    y : массив/список правильных значений размера n_samples
    n_epoch : количество эпох обучения (default = 1)
    '''    
    def __init__(self, X, y, n_epoch=1):
        self.X = np.array(X)
        self.y = np.array(y)
        self.n_epoch = n_epoch
        self.i = -1
        self.epoch = 0
    
    def __iter__(self):
        '''Нужно для использования итератора в цикле for
        Здесь ничего менять не надо
        '''
        return self

    def __next__(self):
        '''Выдача следующего примера.
        
        Выход
        -------
        Метод возвращает очередной пример как из X, так и из y
        '''
        self.i += 1
        if self.i >= self.X.shape[0]:
            self.i = 0
            self.epoch += 1
        if self.epoch >= self.n_epoch:
            raise StopIteration
        return self.X[self.i], self.y[self.i]

### Пример использования

In [7]:
np.random.seed(0)
n_train_samples = 1000
n_test_samples = 500
n_features = 15

trainX = np.append(np.ones((n_train_samples, 1)), np.random.rand(n_train_samples, n_features), axis=1)
W = np.random.rand(n_features + 1)
trainY = trainX @ W

testX = np.append(np.ones((n_train_samples, 1)), np.random.rand(n_train_samples, n_features), axis=1)
testY = testX @ W

n_epoch = 2
l1_coef = 0.0000000001
l2_coef = 0.0000000001
delta = 0.000000001
alpha = 0.13311

trainiterator = TrainIterator(trainX, trainY, n_epoch)
loss = Loss(l1_coef, l2_coef)
grad = Grad(loss, delta)
sgd = SGD(grad, alpha)
reg = Regression(sgd, trainiterator, n_epoch)
reg.fit(trainX, trainY)
y_pred = reg.predict(testX)
acc = reg.score(testY, y_pred)
print('Accuracy is %s' % str(acc))

Accuracy is 0.9999999996161648


### Покройте ваш класс тестами

#### Импортированный тест регрессии

In [8]:
trainX, trainY, testX, testY = generate_regression_data(Nfeat=100, Mtrain=150, Mtest=150)

In [21]:
n_epoch = 7
delta = 1e-07
l1_coef = 4.166626769164239e-10
l2_coef = -1.1334034790699833e-09
alpha = 0.022233325294654372

trainiterator = TrainIterator(trainX, trainY, n_epoch)
loss = Loss(l1_coef, l2_coef)
grad = Grad(loss, delta)
sgd = SGD(grad, alpha)
reg = Regression(sgd, trainiterator, n_epoch)

test_regression_model(reg, trainX, trainY, testX, testY)

TEST REGRESSION MODEL: Your accuracy is 0.9525424573474773


### Подберите оптимальные параметры для вашей модели
Здесь ниже указаны параметры для запуска на тесте `test_regression_model` и данных `generate_regression_data(Nfeat=100, Mtrain=150, Mtest=150)` при `np.random.seed(0)`.

Если считать тройку (`l1_coef`, `l2_coef`, `alpha`) точкой `prm` в пространстве, а действия [обучение, предсказание, подсчёт точности] считать функцией `regr` (значением которой является accuracy), зависящей от параметров `l1_coef`, `l2_coef`, `alpha`, то можно итеративно вычислять градиент от `regr` в точке `prm`, двигать точку `prm` в направлении этого градиента, тем самым приближать функцию `regr` к наибольшему значению. В качестве начального приближения для `prm` выбирается точка (0, 0, 0).

У этого способа также имеется собственный параметр `eps=1e-09`, шаг подъёма, выбранный из соображений, что большее 1e-08 и меньшее 1e-10 либо требуют очень много итераций для приближения, либо дают худшую точность.

Функция `prm` ресурсозатратная, вычисляется долго, в зависимости от `n_epoch` и объёма тренировочных данных. За одну итерацию для сдвига точки `prm` функция `regr` вычисляется 4 раза. Варьирование параметров `delta`, `n_epoch` производится вручную, в то время как подбор параметров `l1_coef`, `l2_coef`, `alpha` производится вышеописанным "градиентным подъёмом" автоматически.

#### Подбор `delta`
При варьировании параметра `delta` в промежутке [1e-07, 1e-03] и при фиксированном `n_epoch` не наблюдалось значительного изменения score (разница составляла не более 0.00003).

#### Подбор `n_epoch`
При варьировании параметра `n_epoch` в диапазоне {1, ..., 8} при `delta=1e-07` были получены следующие точности:

`n_epoch=8`, `score=0.9385284009811397`

`n_epoch=7`, `score=0.9525424537366032`

`n_epoch=6`, `score=0.9437277416335921`

`n_epoch=5`, `score=0.9301497096183774`

`n_epoch=4`, `score=0.9117210979400182`

`n_epoch=3`, `score=0.8860629826647402`

`n_epoch=2`, `score=0.8476816327041838`

`n_epoch=1`, `score=0.7688739944173726` (требует, видимо, очень много времени или более мощной машины)

In [20]:
# Нужно подобрать alpha, delta, l1_coef, l2_coef, n_epoch для максимизации score

delta = 1e-07
n_epoch = 7


def regr(prm):
    trainiterator = TrainIterator(trainX, trainY, n_epoch)
    loss = Loss(prm[0], prm[1])
    grad = Grad(loss, delta)
    sgd = SGD(grad, prm[2])
    reg = Regression(sgd, trainiterator, n_epoch)
    reg.fit(trainX, trainY)
    y_pred = reg.predict(testX)
    return reg.score(testY, y_pred)


def gradient(prm, fixed):
    grd = np.zeros(3)
    for i, e in enumerate(np.eye(3) * 0.000001):
        grd[i] = (regr(prm + e) - fixed) / 0.000001
    return grd


prm = np.array([0, 0, 0])
score_old = 1
score = 0
eps = 1e-09

while score < 0.8:
    score = regr(prm)
    prm = prm + gradient(prm, score) * eps
    score_old = score
    print('iteration score:', score)
    
print('final score:', score)
print('l1_coef:', prm[0])
print('l2_coef:', prm[1])
print('alpha:', prm[2])
print('delta:', delta)
print('n_epoch:', n_epoch)

iteration score: -214.2945085757396
iteration score: 0.9525424537366032
final score: 0.9525424537366032
l1_coef: 4.166626769164239e-10
l2_coef: -1.1334034790699833e-09
alpha: 0.022233325294654372
delta: 1e-07
n_epoch: 7
